In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Charger les données
df = pd.read_csv('geo_safety_data.csv')

# Encoder les variables catégorielles
label_encoder = LabelEncoder()
df['Vict Sex'] = label_encoder.fit_transform(df['Vict Sex'])  # Exemple : 'F' -> 0, 'M' -> 1

# Si tu as d'autres variables catégorielles comme 'lieu', tu peux les encoder ainsi
df = pd.get_dummies(df, columns=['LOCATION'])  # Remplace 'lieu' par la colonne appropriée

# Séparer les caractéristiques (X) et la cible (y)
X = df.drop('danger', axis=1)  # Les colonnes d'entrée
y = df['danger']  # La colonne cible

# Séparer en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normaliser les données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Créer et entraîner le modèle
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Faire des prédictions
y_pred = model.predict(X_test)

# Évaluer le modèle
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9617021276595744
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95        99
           1       0.96      0.97      0.97       136

    accuracy                           0.96       235
   macro avg       0.96      0.96      0.96       235
weighted avg       0.96      0.96      0.96       235



In [11]:
def predict_danger(model, scaler, label_encoder, input_data):
    # Encoder 'Vict Sex' et 'LOCATION' (si ce n'est pas déjà fait pendant l'entraînement)
    input_data[1] = label_encoder.transform([input_data[1]])[0]  # Encodage du sexe (F/M)
    input_data[7] = label_encoder.transform([input_data[7]])[0]  # Encodage de la location (ex: "souk", "plage", etc.)
    
    # Appliquer le StandardScaler aux données (si le modèle a été normalisé pendant l'entraînement)
    input_data = scaler.transform([input_data])
    
    # Faire la prédiction
    prediction = model.predict(input_data)
    
    return prediction[0]


In [13]:

# Exemple de données d'entrée (remplir selon les valeurs réelles)
input_data = [25, 'F', 14, 12, 8, 33.598128973340515, -7.664624676533552]  # Exemple pour Vict Age, Vict Sex, heure, jour, mois, LAT, LON, LOCATION

# Effectuer la prédiction
prediction = predict_danger(model, scaler, label_encoder, input_data)

# Afficher le résultat
print("Le niveau de danger est : ", "DANGEREUX" if prediction == 1 else "SÛR")

IndexError: list index out of range